In [5]:
from utils.kernel import LaplacianKernel

import math
import warnings

import torch
import gpytorch
from gpytorch.kernels import RBFKernel
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd




%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
init_lengthscale = RBFKernel().lengthscale.item()
init_lengthscale

0.6931471824645996

In [3]:
import torch
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_model
from botorch.acquisition.analytic import ExpectedImprovement
from botorch.optim import optimize_acqf

# 1. Basic BO

- 目的関数: sphere function
- 代理モデル: `botorch.models.SingleTaskGP`
- 獲得関数: `botorch.acquisition.analitic.ExpectedImprovement`

In [34]:
def sphere_function(x):
    return torch.sum(x ** 2, 1, keepdim=True)

def generate_initial_data(target_function, shape=(10,1)):
    train_x = torch.rand(shape).double()
    train_y = target_function(train_x)
    # best_val = train_y.max().item()
    best_val = train_y.min().item()
    return train_x, train_y, best_val

def get_next_points(surrogate, x_train, y_train, best_y, bounds, n_points=1):
    y_train = - y_train

    model = surrogate(x_train, y_train)
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll)
    
    acq_func = ExpectedImprovement(model=model, best_f=best_y)

    candidates, _ = optimize_acqf(acq_function=acq_func,
                                  bounds=bounds,
                                  q=n_points,
                                  num_restarts=5,
                                  raw_samples=20)
    
    return candidates


def run_bo(n_runs, 
           dim, 
           init_eval_num,
           target_function, 
           surrogate):
    
    init_x, init_y, best_init_y = generate_initial_data(target_function,
                                                        shape=(init_eval_num, dim))
    # ここは目的関数の定義域と正解の位置によって変えなければいけない
    bounds = torch.stack([-torch.ones(dim), torch.ones(dim)]).double()

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        for i in range(n_runs):
            print(f'Iter: {i+1}')

            new_candidates = get_next_points(surrogate, init_x, init_y, best_init_y, bounds, n_points=1)

            print(new_candidates)

            new_results = target_function(new_candidates)
            
            print(f'New candidates: {new_candidates}')           
            init_x = torch.cat([init_x, new_candidates])
            init_y = torch.cat([init_y, new_results])

            # best_init_y = init_y.max().item()
            best_init_y = init_y.min().item()
            print(f'Current best y: {best_init_y}')
            print()



n_runs = 100
dim = 5
init_eval_num = 10
target_func = sphere_function
surrogate = SingleTaskGP

run_bo(n_runs, dim, init_eval_num, target_func, surrogate)




# n_runs = 100
# dim = 5
# target_func = sphere_function
# init_x, init_y, best_init_y = generate_initial_data(target_func, shape=(10, dim))
# bounds = torch.stack([-torch.ones(dim), torch.ones(dim)]).double()

# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore")
#     for i in range(n_runs):
#         print(f'Iter: {i+1}')

#         surrogate = SingleTaskGP
#         new_candidates = get_next_points(surrogate, init_x, init_y, best_init_y, bounds, n_points=1)

#         print(new_candidates)

#         new_results = target_func(new_candidates)
        
#         print(f'New candidates: {new_candidates}')           
#         init_x = torch.cat([init_x, new_candidates])
#         init_y = torch.cat([init_y, new_results])

#         # best_init_y = init_y.max().item()
#         best_init_y = init_y.min().item()
#         print(f'Current best y: {best_init_y}')
#         print()

Iter: 1
tensor([[0.3717, 0.2553, 0.1458, 0.0350, 0.3506]], dtype=torch.float64)
New candidates: tensor([[0.3717, 0.2553, 0.1458, 0.0350, 0.3506]], dtype=torch.float64)
Current best y: 0.3486996169430295

Iter: 2
tensor([[ 0.4563,  0.4167, -0.1039,  0.1661,  0.1952]], dtype=torch.float64)
New candidates: tensor([[ 0.4563,  0.4167, -0.1039,  0.1661,  0.1952]], dtype=torch.float64)
Current best y: 0.3486996169430295

Iter: 3
tensor([[ 0.5190,  0.3052, -0.1297, -0.1520,  0.4959]], dtype=torch.float64)
New candidates: tensor([[ 0.5190,  0.3052, -0.1297, -0.1520,  0.4959]], dtype=torch.float64)
Current best y: 0.3486996169430295

Iter: 4
tensor([[ 0.4382,  0.3407,  0.1047, -0.2244, -0.0206]], dtype=torch.float64)
New candidates: tensor([[ 0.4382,  0.3407,  0.1047, -0.2244, -0.0206]], dtype=torch.float64)
Current best y: 0.3486996169430295

Iter: 5
tensor([[ 0.3636, -0.1125, -0.0475, -0.0041,  0.0132]], dtype=torch.float64)
New candidates: tensor([[ 0.3636, -0.1125, -0.0475, -0.0041,  0.0132]

In [ ]:
columns = ['（関数の）評価数', '評価点', '評価値', '最適値']

In [10]:
pd.DataFrame([init_x, init_y])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 10) + inhomogeneous part.

In [6]:
init_x, init_y, best_init_y = generate_initial_data(target_func, shape=(10, dim))

In [8]:
init_y

tensor([[0.9017],
        [0.7108],
        [2.0545],
        [0.0713],
        [0.4567],
        [1.8403],
        [1.7546],
        [0.4067],
        [2.1060],
        [0.3794]], dtype=torch.float64)

In [9]:
best_init_y

0.0712500359186734

In [31]:
[(best_init_y) for i in range(len(init_x))]

[0.0712500359186734,
 0.0712500359186734,
 0.0712500359186734,
 0.0712500359186734,
 0.0712500359186734,
 0.0712500359186734,
 0.0712500359186734,
 0.0712500359186734,
 0.0712500359186734,
 0.0712500359186734]

0.0712500359186734

tensor([[0.9017],
        [0.7108],
        [2.0545],
        [0.0713],
        [0.4567],
        [1.8403],
        [1.7546],
        [0.4067],
        [2.1060],
        [0.3794]], dtype=torch.float64)

In [39]:
res_dict = {}
res_dict['評価点'] = [tuple(x.numpy()) for x in init_x]
res_dict['評価値'] = [y.numpy()[0] for y in init_y]
res_dict['最適値'] = [best_init_y if value == best_init_y else np.nan for value in init_y]
pd.DataFrame(res_dict)

,評価点,評価値,最適値
0,"(0.4787622094154358, 0.6087285876274109, 0.289...",0.901684,NaN
1,"(0.19345933198928833, 0.3063541054725647, 0.55...",0.710838,NaN
2,"(0.82276850938797, 0.8917384743690491, 0.47172...",2.054473,NaN
3,"(0.1239517331123352, 0.12101805210113525, 0.01...",0.071250,0.07125
4,"(0.13844722509384155, 0.5720028877258301, 0.20...",0.456715,NaN
5,"(0.6539438366889954, 0.9334130883216858, 0.458...",1.840254,NaN
6,"(0.7657948136329651, 0.5886203646659851, 0.174...",1.754612,NaN
7,"(0.013333737850189209, 0.46968162059783936, 0....",0.406734,NaN
8,"(0.8907178044319153, 0.7338572144508362, 0.175...",2.105998,NaN
9,"(0.010488688945770264, 0.2895098328590393, 0.0...",0.379447,NaN


In [ ]:

res_dict['最適値'] = [best_init_y if value == best_init_y else np.nan for value in init_y]


,評価点,評価値
0,"(0.4787622094154358, 0.6087285876274109, 0.289...","(0.901684142829847,)"
1,"(0.19345933198928833, 0.3063541054725647, 0.55...","(0.7108382559391764,)"
2,"(0.82276850938797, 0.8917384743690491, 0.47172...","(2.0544728738820055,)"
3,"(0.1239517331123352, 0.12101805210113525, 0.01...","(0.0712500359186734,)"
4,"(0.13844722509384155, 0.5720028877258301, 0.20...","(0.4567145676058111,)"
5,"(0.6539438366889954, 0.9334130883216858, 0.458...","(1.8402538302347295,)"
6,"(0.7657948136329651, 0.5886203646659851, 0.174...","(1.7546116301407828,)"
7,"(0.013333737850189209, 0.46968162059783936, 0....","(0.40673350341078773,)"
8,"(0.8907178044319153, 0.7338572144508362, 0.175...","(2.1059981532401864,)"
9,"(0.010488688945770264, 0.2895098328590393, 0.0...","(0.3794468931781694,)"


In [18]:
pd.DataFrame([[tuple(x.numpy()) for x in init_x], [tuple(y.numpy()) for y in init_y]])

,0,1,2,3,4,5,6,7,8,9
0,"(0.4787622094154358, 0.6087285876274109, 0.289...","(0.19345933198928833, 0.3063541054725647, 0.55...","(0.82276850938797, 0.8917384743690491, 0.47172...","(0.1239517331123352, 0.12101805210113525, 0.01...","(0.13844722509384155, 0.5720028877258301, 0.20...","(0.6539438366889954, 0.9334130883216858, 0.458...","(0.7657948136329651, 0.5886203646659851, 0.174...","(0.013333737850189209, 0.46968162059783936, 0....","(0.8907178044319153, 0.7338572144508362, 0.175...","(0.010488688945770264, 0.2895098328590393, 0.0..."
1,"(0.901684142829847,)","(0.7108382559391764,)","(2.0544728738820055,)","(0.0712500359186734,)","(0.4567145676058111,)","(1.8402538302347295,)","(1.7546116301407828,)","(0.40673350341078773,)","(2.1059981532401864,)","(0.3794468931781694,)"


In [16]:
for i in init_x:
    print(tuple(i.numpy()))

(0.4787622094154358, 0.6087285876274109, 0.28994810581207275, 0.25772714614868164, 0.38913649320602417)
(0.19345933198928833, 0.3063541054725647, 0.5537847280502319, 0.3794631361961365, 0.35901129245758057)
(0.82276850938797, 0.8917384743690491, 0.47172480821609497, 0.14729923009872437, 0.5814688205718994)
(0.1239517331123352, 0.12101805210113525, 0.014203846454620361, 0.157700777053833, 0.1271587610244751)
(0.13844722509384155, 0.5720028877258301, 0.20304524898529053, 0.16154998540878296, 0.20744603872299194)
(0.6539438366889954, 0.9334130883216858, 0.4587429165840149, 0.5251654386520386, 0.23474985361099243)
(0.7657948136329651, 0.5886203646659851, 0.17435288429260254, 0.1835760474205017, 0.8704004287719727)
(0.013333737850189209, 0.46968162059783936, 0.3411286473274231, 0.11811220645904541, 0.2358720898628235)
(0.8907178044319153, 0.7338572144508362, 0.17532563209533691, 0.6337732076644897, 0.584522008895874)
(0.010488688945770264, 0.2895098328590393, 0.08960634469985962, 0.29120594

# 2. BO rbf

- 目的関数: sphere function
- 代理モデル: Exact GP (RBF カーネル)
- 獲得関数: `botorch.acquisition.analitic.ExpectedImprovement`

In [29]:
from botorch.models.gpytorch import GPyTorchModel
from botorch.utils.datasets import SupervisedDataset
from gpytorch.distributions import MultivariateNormal
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.means import ConstantMean
from gpytorch.models import ExactGP


class SimpleCustomGP(ExactGP, GPyTorchModel):

    _num_outputs = 1  # to inform GPyTorchModel API

    def __init__(self, train_X, train_Y):
        # squeeze output dim before passing train_Y to ExactGP
        super().__init__(train_X, train_Y.squeeze(-1), GaussianLikelihood())
        self.mean_module = ConstantMean()
        self.covar_module = ScaleKernel(
            base_kernel=RBFKernel(ard_num_dims=train_X.shape[-1]),
        )
        self.to(train_X)  # make sure we're on the right device/dtype

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

In [33]:
def sphere_function(x):
    return torch.sum(x ** 2, 1, keepdim=True)

def generate_initial_data(target_function, shape=(10,1)):
    train_x = torch.rand(shape).double()
    train_y = target_function(train_x)
    # best_val = train_y.max().item()
    best_val = train_y.min().item()
    return train_x, train_y, best_val

def get_next_points(x_train, y_train, best_y, bounds, n_points=1):
    y_train = - y_train

    model = SimpleCustomGP(x_train, y_train)
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll)
    
    acq_func = ExpectedImprovement(model=model, best_f=best_y)

    candidates, _ = optimize_acqf(acq_function=acq_func,
                                  bounds=bounds,
                                  q=n_points,
                                  num_restarts=5,
                                  raw_samples=20)
    
    return candidates


n_runs = 100
dim = 5
target_func = sphere_function
init_x, init_y, best_init_y = generate_initial_data(target_func, shape=(10, dim))
bounds = torch.stack([-torch.ones(dim), torch.ones(dim)]).double()

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    for i in range(n_runs):
        print(f'Iter: {i+1}')

        new_candidates = get_next_points(init_x, init_y, best_init_y, bounds, n_points=1)

        print(new_candidates)

        new_results = target_func(new_candidates)
        
        print(f'New candidates: {new_candidates}')           
        init_x = torch.cat([init_x, new_candidates])
        init_y = torch.cat([init_y, new_results])

        # best_init_y = init_y.max().item()
        best_init_y = init_y.min().item()
        print(f'Current best y: {best_init_y}')
        print()

Iter: 1
tensor([[ 0.0188, -0.9695,  0.2354, -1.0000, -0.3466]], dtype=torch.float64)
New candidates: tensor([[ 0.0188, -0.9695,  0.2354, -1.0000, -0.3466]], dtype=torch.float64)
Current best y: 0.09332987277060312

Iter: 2
tensor([[-0.5268,  0.2444,  0.4532, -0.7255, -0.0798]], dtype=torch.float64)
New candidates: tensor([[-0.5268,  0.2444,  0.4532, -0.7255, -0.0798]], dtype=torch.float64)
Current best y: 0.09332987277060312

Iter: 3
tensor([[ 0.3714, -0.1850, -0.4345,  0.5051, -0.3159]], dtype=torch.float64)
New candidates: tensor([[ 0.3714, -0.1850, -0.4345,  0.5051, -0.3159]], dtype=torch.float64)
Current best y: 0.09332987277060312

Iter: 4
tensor([[-0.5714, -0.1240,  0.0936,  0.3213,  0.2112]], dtype=torch.float64)
New candidates: tensor([[-0.5714, -0.1240,  0.0936,  0.3213,  0.2112]], dtype=torch.float64)
Current best y: 0.09332987277060312

Iter: 5
tensor([[-0.2370,  0.3616,  0.3430,  0.6788, -0.8161]], dtype=torch.float64)
New candidates: tensor([[-0.2370,  0.3616,  0.3430,  0.

In [34]:
model

SimpleCustomGP(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (mean_module): ConstantMean()
  (covar_module): ScaleKernel(
    (base_kernel): RBFKernel(
      (raw_lengthscale_constraint): Positive()
    )
    (raw_outputscale_constraint): Positive()
  )
)

In [35]:
model.likelihood

GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

# 3. BO Laplacian

- 目的関数: sphere function
- 代理モデル: Exact GP (Laplacian カーネル)
- 獲得関数: `botorch.acquisition.analitic.ExpectedImprovement`

In [39]:
from botorch.models.gpytorch import GPyTorchModel
from botorch.utils.datasets import SupervisedDataset
from gpytorch.distributions import MultivariateNormal
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.means import ConstantMean
from gpytorch.models import ExactGP


class SimpleCustomGP(ExactGP, GPyTorchModel):

    _num_outputs = 1  # to inform GPyTorchModel API

    def __init__(self, train_X, train_Y):
        # squeeze output dim before passing train_Y to ExactGP
        super().__init__(train_X, train_Y.squeeze(-1), GaussianLikelihood())
        self.mean_module = ConstantMean()
        self.covar_module = ScaleKernel(
            base_kernel=LaplacianKernel(ard_num_dims=train_X.shape[-1]),
        )
        self.to(train_X)  # make sure we're on the right device/dtype

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

In [47]:
def sphere_function(x):
    return torch.sum(x ** 2, 1, keepdim=True)

def generate_initial_data(target_function, shape=(10,1)):
    train_x = torch.rand(shape).double()
    train_y = target_function(train_x)
    # best_val = train_y.max().item()
    best_val = train_y.min().item()
    return train_x, train_y, best_val

def get_next_points(x_train, y_train, best_y, bounds, n_points=1):
    y_train = - y_train

    # model = SingleTaskGP(x_train, y_train)
    model = SimpleCustomGP(x_train, y_train)

    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll)
    
    acq_func = ExpectedImprovement(model=model, best_f=best_y)

    candidates, _ = optimize_acqf(acq_function=acq_func,
                                  bounds=bounds,
                                  q=n_points,
                                  num_restarts=5,
                                  raw_samples=20)
    
    return candidates


n_runs = 100
dim = 5
target_func = sphere_function
init_x, init_y, best_init_y = generate_initial_data(target_func, shape=(10, dim))
bounds = torch.stack([-torch.ones(dim), torch.ones(dim)]).double()

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    for i in range(n_runs):
        print(f'Iter: {i+1}')

        new_candidates = get_next_points(init_x, init_y, best_init_y, bounds, n_points=1)

        new_results = target_func(new_candidates)
        
        print(f'New candidates: {new_candidates}')
        init_x = torch.cat([init_x, new_candidates])
        init_y = torch.cat([init_y, new_results])

        # best_init_y = init_y.max().item()
        best_init_y = init_y.min().item()
        print(f'Current best y: {best_init_y}')
        print()

Iter: 1
New candidates: tensor([[-0.6172, -0.2208, -0.4988, -1.0000, -0.5797]], dtype=torch.float64)
Current best y: 1.1278744939184762

Iter: 2
New candidates: tensor([[-1.0000, -1.0000, -0.9995,  0.9982,  1.0000]], dtype=torch.float64)
Current best y: 1.1278744939184762

Iter: 3
New candidates: tensor([[-0.3403, -1.0000,  1.0000,  1.0000, -1.0000]], dtype=torch.float64)
Current best y: 1.1278744939184762

Iter: 4
New candidates: tensor([[ 1.0000,  1.0000, -1.0000, -0.3864, -1.0000]], dtype=torch.float64)
Current best y: 1.1278744939184762

Iter: 5
New candidates: tensor([[-1.0000,  1.0000,  1.0000, -0.6412, -1.0000]], dtype=torch.float64)
Current best y: 1.1278744939184762

Iter: 6
New candidates: tensor([[ 0.6835, -1.0000, -0.6772, -0.1746,  0.1351]], dtype=torch.float64)
Current best y: 1.1278744939184762

Iter: 7
New candidates: tensor([[-1.0000, -1.0000,  1.0000, -1.0000,  0.8317]], dtype=torch.float64)
Current best y: 1.1278744939184762

Iter: 8
New candidates: tensor([[-1.0000,

In [44]:
x_train, y_train, _ = generate_initial_data(sphere_function)

model = SimpleCustomGP(x_train, y_train)

In [45]:
model.likelihood

GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

In [46]:
model

SimpleCustomGP(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (mean_module): ConstantMean()
  (covar_module): ScaleKernel(
    (base_kernel): LaplacianKernel(
      (raw_lengthscale_constraint): Positive()
    )
    (raw_outputscale_constraint): Positive()
  )
)